In [1]:
import collections
import re
from d2l import torch as d2l

In [2]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [4]:
def tokenize(lines, token='word'):
    if token=='word':
        return [line.split() for line in lines]
    elif token=='char':
        return [list(line) for line in lines]
    else:
        print("Error: unknow token type!!!")
        return ""

tokens=tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [5]:
class Vocab:
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [6]:
count_corpus(["a", "a", "b", "c"])

Counter({'a': 2, 'b': 1, 'c': 1})

In [7]:
count_corpus([["a", "a", "b", "c"], ["a", "d"]])

Counter({'a': 3, 'b': 1, 'c': 1, 'd': 1})

In [8]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [9]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [10]:
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)

In [11]:
corpus

[3,
 9,
 2,
 1,
 3,
 5,
 13,
 2,
 1,
 13,
 4,
 15,
 9,
 5,
 6,
 2,
 1,
 21,
 19,
 1,
 9,
 1,
 18,
 1,
 17,
 2,
 12,
 12,
 8,
 5,
 3,
 9,
 2,
 1,
 3,
 5,
 13,
 2,
 1,
 3,
 10,
 4,
 22,
 2,
 12,
 12,
 2,
 10,
 1,
 16,
 7,
 10,
 1,
 8,
 7,
 1,
 5,
 3,
 1,
 17,
 5,
 12,
 12,
 1,
 21,
 2,
 1,
 15,
 7,
 6,
 22,
 2,
 6,
 5,
 2,
 6,
 3,
 1,
 3,
 7,
 1,
 8,
 20,
 2,
 4,
 23,
 1,
 7,
 16,
 1,
 9,
 5,
 13,
 17,
 4,
 8,
 1,
 2,
 24,
 20,
 7,
 14,
 6,
 11,
 5,
 6,
 18,
 1,
 4,
 1,
 10,
 2,
 15,
 7,
 6,
 11,
 5,
 3,
 2,
 1,
 13,
 4,
 3,
 3,
 2,
 10,
 1,
 3,
 7,
 1,
 14,
 8,
 1,
 9,
 5,
 8,
 1,
 18,
 10,
 2,
 19,
 1,
 2,
 19,
 2,
 8,
 1,
 8,
 9,
 7,
 6,
 2,
 1,
 4,
 6,
 11,
 3,
 17,
 5,
 6,
 23,
 12,
 2,
 11,
 1,
 4,
 6,
 11,
 1,
 9,
 5,
 8,
 1,
 14,
 8,
 14,
 4,
 12,
 12,
 19,
 1,
 20,
 4,
 12,
 2,
 1,
 16,
 4,
 15,
 2,
 1,
 17,
 4,
 8,
 1,
 16,
 12,
 14,
 8,
 9,
 2,
 11,
 1,
 4,
 6,
 11,
 1,
 4,
 6,
 5,
 13,
 4,
 3,
 2,
 11,
 1,
 3,
 9,
 2,
 16,
 5,
 10,
 2,
 1,
 21,
 14,
 10,
 6,
 2,
 11,
 1,
 21,